## Convolutional Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN model
def build_custom_cnn(input_shape=(32, 32, 3), num_classes=10):
    model = models.Sequential()

    # Convolutional layer 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional layer 2
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional layer 3
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    # Flatten the tensor output
    model.add(layers.Flatten())

    # Dense layer
    model.add(layers.Dense(64, activation='relu'))

    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Example usage:
model = build_custom_cnn(input_shape=(32, 32, 3), num_classes=10)
model.summary()

# Compilation
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# (Optional) Training code example
# Assuming you have training_data and training_labels:
# model.fit(training_data, training_labels, epochs=10, batch_size=32)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

### Recurrent Neural Network (RNN)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Define constants
input_sequence_length = 10  # Example sequence length, modify as needed
input_features = 1          # Number of input features
output_features = 1         # Number of output features
hidden_units = 50           # Number of hidden units in the RNN layer

# Build the model
model = Sequential()
model.add(SimpleRNN(hidden_units, activation='tanh', input_shape=(input_sequence_length, input_features), return_sequences=True))
model.add(Dense(output_features, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Model summary
model.summary()

# Train the model
# For this, you'll need training data. Here's an example of dummy data:
X_train = np.random.randn(1000, input_sequence_length, input_features)
Y_train = np.random.randn(1000, input_sequence_length, output_features)
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)

#Predict with the model
prediction = model.predict(np.random.randn(1, input_sequence_length, input_features))


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 10, 50)            2600      
_________________________________________________________________
dense_2 (Dense)              (None, 10, 1)             51        
Total params: 2,651
Trainable params: 2,651
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
25/25 [==============================] - 3s 15ms/step - loss: 1.0413 - val_loss: 1.0070
Epoch 2/10
25/25 [==============================] - 0s 8ms/step - loss: 1.0225 - val_loss: 1.0072
Epoch 3/10
25/25 [==============================] - 0s 8ms/step - loss: 1.0196 - val_loss: 1.0044
Epoch 4/10
25/25 [==============================] - 0s 8ms/step - loss: 1.0204 - val_loss: 1.0130
Epoch 5/10
25/25 [==============================] - 0s 7ms/step - loss: 1.0160 - val_loss: 1.0046
Epoch 6/10
25/25 [=====

## Radial Basis Function Network (RBFN)

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
tf.compat.v1.disable_eager_execution()
# Generate some sample data
data = np.random.rand(100, 2)
labels = np.sin(data[:, 0] + data[:, 1])

# Standardize the data
scaler = StandardScaler()
data = scaler.fit_transform(data)

# Use K-Means clustering to find RBF centers
kmeans = KMeans(n_clusters=10, random_state=0).fit(data)
centers = kmeans.cluster_centers_

# Create a TensorFlow graph
input_placeholder = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, 2])
centers_placeholder = tf.compat.v1.placeholder(dtype=tf.float32, shape=[10, 2])
betas_placeholder = tf.compat.v1.placeholder(dtype=tf.float32, shape=[10])

# Calculate the distance between data points and RBF centers
distances = tf.norm(input_placeholder[:, tf.newaxis] - centers_placeholder, axis=2, ord=2)

# Define the RBF layer
rbf_layer = tf.exp(-betas_placeholder * distances)

# Create the output layer (fully connected)
weights = tf.compat.v1.Variable(tf.compat.v1.random_normal([10, 1]))
biases = tf.compat.v1.Variable(tf.compat.v1.random_normal([1]))
output = tf.compat.v1.matmul(rbf_layer, weights) + biases

# Define the target values
target = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, 1])

# Define the loss function (MSE)
loss = tf.reduce_mean(tf.square(target - output))

# Define the optimizer
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(loss)

# Create a TensorFlow session and train the RBFN
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())

    for _ in range(1000):
        feed_dict = {
            input_placeholder: data,
            centers_placeholder: centers,
            betas_placeholder: np.ones(10),
            target: labels.reshape(-1, 1)
        }
        _, current_loss = sess.run([train_op, loss], feed_dict=feed_dict)
        print("Loss:", current_loss)

    # Evaluate the RBFN
    test_data = np.random.rand(10, 2)
    test_data = scaler.transform(test_data)
    feed_dict = {
        input_placeholder: test_data,
        centers_placeholder: centers,
        betas_placeholder: np.ones(10)
    }
    predictions = sess.run(output, feed_dict=feed_dict)

    print("Predictions:", predictions)


c:\Users\deepn\anaconda3\envs\data\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\deepn\anaconda3\envs\data\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Loss: 1.2158515
Loss: 0.6635001
Loss: 0.40345496
Loss: 0.28007412
Loss: 0.22061619
Loss: 0.19108361
Loss: 0.17558569
Loss: 0.1666943
Loss: 0.16093613
Loss: 0.15668632
Loss: 0.15318385
Loss: 0.1500718
Loss: 0.14718172
Loss: 0.14443354
Loss: 0.14178866
Loss: 0.13922764
Loss: 0.13674024
Loss: 0.13432051
Loss: 0.13196453
Loss: 0.12966943
Loss: 0.12743284
Loss: 0.12525263
Loss: 0.12312684
Loss: 0.12105373
Loss: 0.11903152
Loss: 0.11705858
Loss: 0.11513332
Loss: 0.11325424
Loss: 0.11141986
Loss: 0.10962878
Loss: 0.107879676
Loss: 0.10617123
Loss: 0.10450219
Loss: 0.10287139
Loss: 0.1012776
Loss: 0.09971979
Loss: 0.09819686
Loss: 0.09670775
Loss: 0.09525149
Loss: 0.09382711
Loss: 0.09243371
Loss: 0.091070384
Loss: 0.08973629
Loss: 0.08843057
Loss: 0.08715245
Loss: 0.08590115
Loss: 0.084675916
Loss: 0.08347606
Loss: 0.082300894
Loss: 0.08114971
Loss: 0.08002192
Loss: 0.078916825
Loss: 0.0778339
Loss: 0.076772526
Loss: 0.075732134
Loss: 0.0747122
Loss: 0.0737122
Loss: 0.07273161
Loss: 0.0717699